#### Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import numpy as np
import time
from tqdm import tqdm
from torch.utils import data
import dgl
import dgl.data
from dgl.dataloading import GraphDataLoader
from dgl.nn import GraphConv
from torch.utils.data import random_split

Using backend: pytorch


#### Define Dataset

In [2]:
dataset = dgl.data.QM9EdgeDataset(label_keys=['mu'], raw_dir="./data")
data_len = len(dataset)

Done loading data from cached files.


In [3]:
print('Number of label:', dataset.num_labels)
count = 0
for graph, labels in dataset:
    print(graph)  # get information of each graph
    print(labels)  # get labels of the corresponding graph
    if count == 5: break
    count += 1


Number of label: 1
Graph(num_nodes=5, num_edges=8,
      ndata_schemes={'pos': Scheme(shape=(3,), dtype=torch.float32), 'attr': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'edge_attr': Scheme(shape=(4,), dtype=torch.float32)})
tensor([0.])
Graph(num_nodes=4, num_edges=6,
      ndata_schemes={'pos': Scheme(shape=(3,), dtype=torch.float32), 'attr': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'edge_attr': Scheme(shape=(4,), dtype=torch.float32)})
tensor([1.6256])
Graph(num_nodes=3, num_edges=4,
      ndata_schemes={'pos': Scheme(shape=(3,), dtype=torch.float32), 'attr': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'edge_attr': Scheme(shape=(4,), dtype=torch.float32)})
tensor([1.8511])
Graph(num_nodes=4, num_edges=6,
      ndata_schemes={'pos': Scheme(shape=(3,), dtype=torch.float32), 'attr': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'edge_attr': Scheme(shape=(4,), dtype=torch.float32)})
tensor([0.])
Graph(num_nod

#### Define Model

In [4]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, o_feats, hidden_dim, out_dim, d_prob=0.15):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, o_feats)
        self.fc1 = nn.Linear(o_feats, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, out_dim)

        self.d_prob = d_prob
    
    def forward(self, g, in_feat, pool_op):
        h = self.conv1(g, in_feat)
        #h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        o = dgl.readout_nodes(graph=g, feat='h', op=pool_op)

        x = F.relu(self.fc1(o))
        x = F.dropout(x, p=self.d_prob)
        x = self.fc2(x)

        return x

#### Training Code

In [5]:
def train(train_loader, epoch, model, optimizer, device, pool_op):
    train_loss = 0.
    for epoch in range(1, epoch+1):
        sum_loss = 0.
        for batched_graph, labels in tqdm(train_loader):
            batched_graph, labels = batched_graph.to(device), labels.to(device)
            pred = model(batched_graph, batched_graph.ndata['attr'].float(), pool_op)
            loss = F.l1_loss(pred, labels)
            sum_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        sum_loss /= len(train_loader)
        #print(f'Epoch: {epoch}, Loss: {sum_loss:.6f}')
        train_loss = sum_loss

    return train_loss

#### Validation and Testing Code

In [6]:
def valid_test(loader, model, device, pool_op):
    final_loss = 0.
    with torch.no_grad():
        for batched_graph, labels in loader:
            batched_graph, labels = batched_graph.to(device), labels.to(device)
            pred = model(batched_graph, batched_graph.ndata['attr'].float(), pool_op)
            loss = F.l1_loss(pred, labels)
            final_loss += loss.item()
        
        final_loss /= len(loader)
        
    return final_loss

#### Set Up Training

In [7]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"using device: {device}")
in_feats = 11
h_feats = 128
o_feats = 64
hidden_dim = 128
out_dim = 1
d_prob = 0.15
learning_rate = 0.01
batch_size = 20
epoch = 5
pool_op = 'sum'
models = []

using device: cuda:0


#### Start Training, Validation

In [8]:
train_losses = []
valid_losses = []
test_sets = []
test_losses = []
for i in range(5):
    train_set, valid_set, test_set = random_split(dataset,
                            [int(data_len*0.8), int(data_len*0.1),
                            data_len-int(data_len*0.8)-int(data_len*0.1)])
    
    model = GCN(in_feats, h_feats, o_feats, hidden_dim, out_dim, d_prob)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model = model.to(device)
    
    train_loader = GraphDataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=8)
    model.train()
    train_loss = train(train_loader, epoch, model, optimizer, device, pool_op)
    print('fold {}, train loss {:.3f}'.format(i+1, train_loss))
    train_losses.append(train_loss)
    
    valid_loader = GraphDataLoader(dataset=valid_set, batch_size=batch_size, shuffle=True, num_workers=8)
    model.eval()
    valid_loss = valid_test(valid_loader, model, device, pool_op)
    print('fold {}, valid loss {:.3f}'.format(i+1, valid_loss))
    valid_losses.append(valid_loss)
    
    test_sets.append(test_set)
    models.append(model)
    
print('average train loss is {:.3f}, std is {:.3f}'.format(np.mean(train_losses), np.std(train_losses)))
print('average validation loss is {:.3f}, std is {:.3f}'.format(np.mean(valid_losses), np.std(valid_losses)))

100%|██████████| 5234/5234 [01:20<00:00, 64.97it/s]

fold 1, train loss 1.158



  0%|          | 0/5234 [00:00<?, ?it/s]

fold 1, valid loss 1.170


100%|██████████| 5234/5234 [01:25<00:00, 61.04it/s]

...


  0%|          | 0/5234 [00:00<?, ?it/s]

fold 4, valid loss 1.158


100%|██████████| 5234/5234 [01:22<00:00, 63.35it/s]

fold 5, train loss 1.161


fold 5, valid loss 1.150
average train loss is 1.160, std is 0.001
average validation loss is 1.158, std is 0.007


#### Start Testing

In [9]:
for i in range(5):
    test_loader = GraphDataLoader(dataset=test_sets[i], batch_size=batch_size, shuffle=True, num_workers=8)
    test_loss = valid_test(test_loader, models[i], device, pool_op)
    test_losses.append(test_loss)

print('average test loss is {:.3f}, std is {:.3f}'.format(np.mean(test_losses), np.std(test_losses)))

average test loss is 1.164, std is 0.005
